In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [2]:
batchsize=100

In [3]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'D:\mrlEyes_2018_01 (1)\Prepared_Data\Train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'D:\mrlEyes_2018_01 (1)\Prepared_Data\Train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 64759 images belonging to 2 classes.
Found 16189 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'D:\mrlEyes_2018_01 (1)\Prepared_Data\Test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 3920 images belonging to 2 classes.


In [5]:
bmodel = VGG19(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 80, 80, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 80, 80, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 80, 80, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 40, 40, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 40, 40, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 40, 40, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 20, 20, 128)       0     

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [9]:
checkpoint = ModelCheckpoint(r'D:\mrlEyes_2018_01 (1)\Prepared_Data\vggmodel',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=50)

C:\Users\lokes\AppData\Local\Temp/ipykernel_2804/567684715.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/50
647/647 [==============================] - ETA: 0s - loss: 0.4381 - accuracy: 0.7936
Epoch 1: val_loss improved from inf to 0.50597, saving model to D:\mrlEyes_2018_01 (1)\Prepared_Data\vggmodel


INFO:tensorflow:Assets written to: D:\mrlEyes_2018_01 (1)\Prepared_Data\vggmodel\assets


INFO:tensorflow:Assets written to: D:\mrlEyes_2018_01 (1)\Prepared_Data\vggmodel\assets


647/647 [==============================] - 2903s 4s/step - loss: 0.4381 - accuracy: 0.7936 - val_loss: 0.5060 - val_accuracy: 0.7628 - lr: 0.0010
Epoch 2/50
647/647 [==============================] - ETA: 0s - loss: 0.3319 - accuracy: 0.8610
Epoch 2: val_loss improved from 0.50597 to 0.44808, saving model to D:\mrlEyes_2018_01 (1)\Prepared_Data\vggmodel


INFO:tensorflow:Assets written to: D:\mrlEyes_2018_01 (1)\Prepared_Data\vggmodel\assets


INFO:tensorflow:Assets written to: D:\mrlEyes_2018_01 (1)\Prepared_Data\vggmodel\assets


647/647 [==============================] - 1795s 3s/step - loss: 0.3319 - accuracy: 0.8610 - val_loss: 0.4481 - val_accuracy: 0.8000 - lr: 0.0010
Epoch 3/50
525/647 [=======================>......] - ETA: 4:41 - loss: 0.3121 - accuracy: 0.8686

In [5]:
from tensorflow.keras.models import load_model

In [6]:
model = load_model(r'D:\mrlEyes_2018_01 (1)\Prepared_Data\models')

In [7]:
#test data
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)

C:\Users\lokes\AppData\Local\Temp/ipykernel_12912/778069267.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


0.7253046035766602
0.8043367266654968


In [8]:
model.save('model1.h5')

In [9]:
model = load_model(r'model1.h5')

In [10]:
#test data
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)

C:\Users\lokes\AppData\Local\Temp/ipykernel_12912/778069267.py:2: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


0.7253046035766602
0.8043367266654968
